## Data download

In [ ]:
#XXX changes
!wget ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!wget ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

!wget ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/relationships_w_pops_121708.txt

In [ ]:
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

# Preparation

In [ ]:
import os
from collections import defaultdict

## Loading HapMap meta-data

In [ ]:
f = open('relationships_w_pops_121708.txt')
pop_ind = defaultdict(list)
f.readline()  # header
offspring = []
for l in f:
    toks = l.rstrip().split('\t')
    fam_id = toks[0]
    ind_id = toks[1]
    mom = toks[2]
    dad = toks[3]
    if mom != '0' or dad != '0':
        offspring.append((fam_id, ind_id))
    pop = toks[-1]
    pop_ind[pop].append((fam_id, ind_id))
f.close()

## Sub-sampling

In [ ]:
os.system('plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap10 --thin 0.1 --geno 0.1')
os.system('plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap1 --thin 0.01 --geno 0.1')

## Getting only autosomal data

In [ ]:
def get_non_auto_SNPs(map_file, exclude_file):
    f = open(map_file)
    w = open(exclude_file, 'w')
    for l in f:
        toks = l.rstrip().split('\t')
        chrom = int(toks[0])
        rs = toks[1]
        if chrom > 22:
            w.write('%s\n' % rs)
    w.close()

In [ ]:
get_non_auto_SNPs('hapmap10.map', 'exclude10.txt')
get_non_auto_SNPs('hapmap1.map', 'exclude1.txt')
#get_non_auto_SNPs('hapmap3_r2_b36_fwd.consensus.qc.poly.map', 'exclude.txt')

In [ ]:
!plink --recode --file hapmap10 --noweb --out hapmap10_auto --exclude exclude10.txt
!plink --recode --file hapmap1 --noweb --out hapmap1_auto --exclude exclude1.txt
#geno!!!
#!plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap_auto --exclude exclude.txt

## Removing offspring

In [ ]:
!plink --file hapmap10_auto --filter-founders --recode --out hapmap10_auto_noofs

## LD-prunning

In [ ]:
!plink --file hapmap10_auto_noofs --indep-pairwise 50 10 0.1 --out keep
!plink --file hapmap10_auto_noofs --extract keep.prune.in --recode --out hapmap10_auto_noofs_ld

## Different coding

In [ ]:
!plink --file hapmap10_auto_noofs_ld --recode12 tab --out hapmap10_auto_noofs_ld_12
!plink --make-bed --file hapmap10_auto_noofs_ld --out hapmap10_auto_noofs_ld

## Single chromosome

In [ ]:
!plink --recode --file hapmap10_auto_noofs --chr 2 --out hapmap10_auto_noofs_2